# Automação Web e Busca de Informações com Python

### Desafio:

Trabalhamos em uma importadora e o preço dos nosso produtos são vinculados a cotação de:
- Dolar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Para isso, vamos criar uma automação web:
- Usaremos o selenium
- Importante baixar o webdriver

## Bibliotecas

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

## Cotação Dolar

In [19]:
navegador = webdriver.Chrome()
# se o chromedriver estiver na mesma pasta da aplicaçao essa linha de código acima vai ficar desse jeito abaixo
# navegador = webdriver.Chrome('chromedriver.exe')
navegador.get('https://www.google.com.br/')

navegador.find_element(
    By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input'
    ).send_keys('cotação dolar')

navegador.find_element(
    By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input'
    ).send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element(
    By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]'
    ).get_attribute('data-value')

navegador.close()

print(cotacao_dolar)

5.520099999999999


## Cotação Euro

In [20]:
navegador = webdriver.Chrome()
navegador.get('https://www.google.com.br/')

navegador.find_element(
    By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input'
    ).send_keys('cotação euro')

navegador.find_element(
    By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input'
    ).send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(
    By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]'
    ).get_attribute('data-value')

navegador.close()

print(cotacao_euro)

6.330299999999999


## Cotação Ouro

In [21]:
navegador = webdriver.Chrome()
navegador.get('https://www.melhorcambio.com/ouro-hoje')

cotacao_ouro = navegador.find_element(
    By.XPATH, '//*[@id="comercial"]'
    ).get_attribute('value')

navegador.close()
cotacao_ouro = cotacao_ouro.replace(',', '.')

print(cotacao_ouro)

323.31


## Atualizando base de preços com as novas cotações
- importanto base de dados
- atualizando preços e o cálculo do preço final

In [22]:
import pandas as pd
# importanto base de dados
tabela = pd.read_excel('Base/Produtos.xlsx')
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [24]:
# calcular novos preços
tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.5201,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6.3303,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.5201,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5.5201,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6.3303,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.5201,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,323.3100,7000.00,1.15,8050.000


In [25]:
# Atualizar colunas
# preço de compra = preço original * cotação
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]
# preço de venda = preço de compra * margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.5201,5520.044799,1.40,7728.062719
1,Carro Renault,4500.00,Euro,6.3303,28486.350000,2.00,56972.700000
2,Notebook Dell,899.99,Dólar,5.5201,4968.034799,1.70,8445.659158
3,IPhone,799.00,Dólar,5.5201,4410.559900,1.70,7497.951830
4,Carro Fiat,3000.00,Euro,6.3303,18990.900000,1.90,36082.710000
5,Celular Xiaomi,480.48,Dólar,5.5201,2652.297648,2.00,5304.595296
6,Joia 20g,20.00,Ouro,323.3100,6466.200000,1.15,7436.130000


## Exportando nova base de preços atualizadas

In [27]:
tabela.to_excel('Base/Produtos_Atualizados.xlsx', index=False)